In [1]:
# %pip install crewai-tools crewai

In [2]:
# %pip install --upgrade langchain_google_vertexai

In [3]:
from langchain_google_vertexai import VertexAI

# To use model
model = VertexAI(model_name="gemini-pro")

In [5]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

FOLDER_NAME="."

In [6]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
# from langchain_vertexai import ChatGemini
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
from vertexai.preview.vision_models import ImageGenerationModel
from langchain_google_vertexai import ChatVertexAI
import uuid, os

# Initialize Gemini LLM
llm = ChatVertexAI(
    model_name='gemini-1.5-pro-002', #"gemini-1.0-pro-002", # Replace with your desired Gemini model
    project_id=os.getenv(PROJECT_ID), # Your Vertex AI project ID
    location="us-central1", # Your Vertex AI location
)


### Step 1: Scraping a website


In [7]:
from crewai_tools import ScrapeWebsiteTool, FileWriterTool, TXTSearchTool
import requests

# Initialize the tool, potentially passing the session
tool = ScrapeWebsiteTool(website_url='https://www.prudential.com.sg/products/health-insurance/medical/prushield')  

# Extract the text
text = tool.run()
print(text)

Using Tool: Read website content
PRUShield – Health Insurance Plan | Get a Quote Online | Prudential Singapore 
Online Payment
Rewards
Personal
Enterprise
Login
We Do
Products
Claims & Services
Priority Programmes
Work With Us
About Us
We Do
We Do Pulse
We Do Hub
We Do Innovation
Our Reason Is You
Products
Health Protection
Life Protection
Wealth Accumulation
Legacy Planning
Buy Insurance Online
Promotions
SRS
Claims & Services
Claims
Payments
Support
Tools
PRUPanel Connect
Priority Programmes
Ascend By Prudential
OPUS By Prudential
PURSUE
PRUVibes
Work With Us
Join as a Financial Rep
Corporate Careers
About Us
About Us
Sustainability
In Our Community
Newsroom
Community Investment
Login
PRUaccess
Corporate Insurance
Rewards
Online Payment
Enterprise
What can we help you with?
Call our PruCustomer Line 1800 333 0333
Call us
Send us an enquiry
Send enquiry
Cancel
Login to
PRUaccess
Corporate Insurance
We Do Pulse
 PRUShoppe
 FAQ
We Do Hub
 Do Health
 Do Wealth
 Do Family
We Do Innovation

### Step2: Write the extracted text to a file


In [8]:
# Initialize the tool
file_writer_tool = FileWriterTool()

# Write content to a file in a specified directory
result = file_writer_tool._run(filename='ai.txt', content = text, directory = '', overwrite=True)
print(result)

An error occurred while writing to the file: 'bool' object has no attribute 'lower'


In [20]:
config=dict(
    llm=dict(
        provider="vertexai",  # Change this to your LLM provider
        config=dict(
            model='gemini-pro' #"llama3.1",  # Specify the model you want to use
        ),
    ),
    embedder=dict(
        provider="vertexai",  # Change this to your LLM provider
        config=dict(
            model="text-embedding-004",  # Specify the embedding model 
        ),
    ),
)

In [21]:
from crewai_tools.tools import TXTSearchTool
rag_tool = TXTSearchTool(txt='./ai.txt',
    config=config, 
    verbose=True)

In [22]:


data_analyst = Agent(
  role='Data Analyst',
  goal='You perfectly know how to analyze any data using provided txt file and searching info via RAG tool',
  backstory='You are data expert',
  verbose=True,
  allow_delegation=False,
  tools=[rag_tool],
    llm=llm
)

test_task = Task(
    description='Show me a Basic health screening packages and vaccination service',
    agent=data_analyst,
    expected_output='A name of company and not much addition information.',
    llm=llm
)




In [23]:
crew = Crew(
    agents=[data_analyst],
    tasks=[test_task]
)

output = crew.kickoff()

2024-11-27 07:51:32,498 - 140434825323712 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Data Analyst
## Task: Show me a company name


# Agent: Data Analyst
## Thought: Thought: I need to search the document for company names.
## Using tool: Search a txt's content
## Tool Input: 
"{\"search_query\": \"company name\"}"
## Tool Output: 
Relevant Content:
Unless prohibited by law, participation in the campaign constitutes permission for Prudential, its advertising and promotional agencies to use any of the Eligible Customer’s names, and/or likeness for advertising and promotional purposes. Each Eligible Customer further agrees and acknowledges that the copyright and all other intellectual property rights in and to all photographs or audio-video or other recordings of the Eligible Customer taken or made in connection with the campaign shall vest solely and absolutely in Prudential without any compensation to the Eligible Customer.
 Prudential may at any time at its absolute discretion, without prior notice or assigning any reason thereof or being liable to any person